# Лабораторная работа №2 «Алгоритм Дойча и алгоритм Дойча–Йожи»


### Вариант 2

Выполнил Проскуряков Роман Владимирович

Реализовать алгоритм Дойча для

f(x)=1;

f(x)=x.

Нарисовать квантовые схемы.

Реализовать алгоритм Дойча–Йожи для

f(x) = x_0  AND x_1;

f(x) = (x_0  ⊕ x_2  ⊕ x_4  ⊕ ...) - XOR только чётных индексов;

f(x) = x ∨ x ̅, где x ̅=NOT x;

f(x) = 1,если старший бит = 1;

f(x) = x ⊕ x:


In [1]:
!pip install qiskit qiskit-aer --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.7 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi
import matplotlib.pyplot as plt

In [3]:
def simulate(qc, shots=1024):
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    print("Результаты измерений:", counts)
    return counts

def draw_and_simulate(qc, title):
    print(f"{title}")
    print(qc.draw(output='text'))  # текстовая визуализация схемы
    return simulate(qc)

def go(qc, title="", shots=1024, noise_model=None):
    print(f"{title}")
    print(qc.draw(output='text'))  # текстовая визуализация схемы
    simulator = AerSimulator(noise_model=noise_model)
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    print("Результаты измерений:", counts)
    return counts

## Алгоритм Дойча

In [4]:
def deutsch_algorithm(oracle_func, title="Deutsch Algorithm"):
  # Создаем схему с 2 кубитами: 1 входной + 1 вспомогательный
  qc = QuantumCircuit(2, 1)

  # Инициализация: вспомогательный кубит в |1⟩, затем Адамар
  qc.x(1)
  qc.h(1)

  # Адамар на входном кубите
  qc.h(0)

  # Оракул f(x)
  oracle_func(qc)

  # Еще один Адамар на входном кубите
  qc.h(0)

  # Измерение входного кубита
  qc.measure(0, 0)
  draw_and_simulate(qc, title)

## Алгоритм Дойча для f(x)=1

| x | f(x) |
|---|---|
| 0 | 1 |
| 1 | 1 |

т.е надо всегда добавлять фазу -1

In [5]:
def oracle_func(qc):
  qc.x(1)

deutsch_algorithm(oracle_func, "Алгоритм Дойча для f(x)=1")

Алгоритм Дойча для f(x)=1
     ┌───┐┌───┐     ┌─┐
q_0: ┤ H ├┤ H ├─────┤M├
     ├───┤├───┤┌───┐└╥┘
q_1: ┤ X ├┤ H ├┤ X ├─╫─
     └───┘└───┘└───┘ ║ 
c: 1/════════════════╩═
                     0 
Результаты измерений: {'0': 1024}


Вывод: функция константная

## Алгоритм Дойча для f(x)=x

| x | f(x) |
|---|---|
| 0 | 0 |
| 1 | 1 |

т.е надо всегда добавлять фазу -1, только когда x==1

In [6]:
def oracle_func(qc):
  qc.cx(0, 1)

deutsch_algorithm(oracle_func, "Алгоритм Дойча для f(x)=x")

Алгоритм Дойча для f(x)=x
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐└───┘└╥┘
q_1: ┤ X ├┤ H ├┤ X ├──────╫─
     └───┘└───┘└───┘      ║ 
c: 1/═════════════════════╩═
                          0 
Результаты измерений: {'1': 1024}


Вывод: функция является сбалансированной

Алгоритм Дойча–Йожи

In [7]:
def deutsch_jozsa_algorithm(oracle_func, n_qubits, title="Deutsch Jozsa Algorithm"):
  """
  Алгоритм Дойча-Йожи для n-битовой функции
  """
  # n входных кубитов + 1 вспомогательный
  qc = QuantumCircuit(n_qubits + 1, n_qubits)

  # Шаг 1: Инициализация вспомогательного кубита в |1⟩
  qc.x(n_qubits)

  # Шаг 2: Применяем Адамары ко всем кубитам
  for i in range(n_qubits + 1):
      qc.h(i)

  # Шаг 3: Применяем оракул
  oracle_func(qc, n_qubits)

  # Шаг 4: Применяем Адамары ко входным кубитам
  for i in range(n_qubits):
      qc.h(i)

  # Шаг 5: Измеряем входные кубиты
  for i in range(n_qubits):
      qc.measure(i, i)

  draw_and_simulate(qc, title)

Алгоритм Дойча–Йожи для f(x) = x_0 AND x_1

| x_0 | x_1 | f(x) |
|---|---|---|
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

т.е надо дбавить фазу -1, только при x_0 == x_1 == 1

In [8]:
def oracle_func_1(qc, n_qubits):
  controls = list(range(n_qubits))  # индексы контролирующих кубитов
  target = n_qubits  # индекс целевого кубита

  qc.mcx(controls, target)

deutsch_jozsa_algorithm(oracle_func_1, 2, "Алгоритм Дойча–Йожи для f(x) = x_0 AND x_1")

Алгоритм Дойча–Йожи для f(x) = x_0 AND x_1
     ┌───┐          ┌───┐┌─┐   
q_0: ┤ H ├───────■──┤ H ├┤M├───
     ├───┤       │  ├───┤└╥┘┌─┐
q_1: ┤ H ├───────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘      ║  ║ 
c: 2/═════════════════════╩══╩═
                          0  1 
Результаты измерений: {'01': 255, '11': 242, '00': 284, '10': 243}


Функция не константная и не сбалансированная

Алгоритм Дойча–Йожи для f(x) = (x_0  ⊕ x_2  ⊕ x_4  ⊕ ...) - XOR только чётных индексов

| x_0 | x_1 | x_2 | x_3 | f(x) |
|-----|-----|-----|-----|------|
|  0  |  ?  |  0  |  ?  |   0  |
|  0  |  ?  |  1  |  ?  |   1  |
|  1  |  ?  |  0  |  ?  |   1  |
|  1  |  ?  |  1  |  ?  |   0  |

| x_0 | x_1 | x_2 | x_3 | x_4 | x_5 | f(x) |
|-----|-----|-----|-----|-----|-----|------|
|  0  |  ?  |  0  |  ?  |  0  |  ?  |   0  |
|  0  |  ?  |  0  |  ?  |  1  |  ?  |   1  |
|  0  |  ?  |  1  |  ?  |  0  |  ?  |   1  |
|  0  |  ?  |  1  |  ?  |  1  |  ?  |   0  |
|  1  |  ?  |  0  |  ?  |  0  |  ?  |   1  |
|  1  |  ?  |  0  |  ?  |  1  |  ?  |   0  |
|  1  |  ?  |  1  |  ?  |  0  |  ?  |   0  |
|  1  |  ?  |  1  |  ?  |  1  |  ?  |   1  |

Будем добавляять фазу на каждой чётной переменной равной 1. Так фаза добавитсялибо чётное количетсво раз - ничего не изменится как и должно или нечётный и всё правильно ведь функция принимает в этом состоянии 1


In [9]:
def oracle_func_2(qc, n_qubits):
  # пройдёмся по всем чётным битам и воспользуемся cx
  for i in range(0, n_qubits, 2):
    qc.cx(i, n_qubits)

deutsch_jozsa_algorithm(oracle_func_2, 4, "Алгоритм Дойча–Йожи для f(x) = (x_0  ⊕ x_2  ⊕ x_4  ⊕ ...) - XOR только чётных индексов")

Алгоритм Дойча–Йожи для f(x) = (x_0  ⊕ x_2  ⊕ x_4  ⊕ ...) - XOR только чётных индексов
     ┌───┐          ┌───┐             ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────────────┤M├───
     ├───┤┌───┐  │  └┬─┬┘             └╥┘   
q_1: ┤ H ├┤ H ├──┼───┤M├───────────────╫────
     ├───┤└───┘  │   └╥┘         ┌───┐ ║ ┌─┐
q_2: ┤ H ├───────┼────╫───────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │    ║  ┌─┐  │  └───┘ ║ └╥┘
q_3: ┤ H ├┤ H ├──┼────╫──┤M├──┼────────╫──╫─
     ├───┤├───┤┌─┴─┐  ║  └╥┘┌─┴─┐      ║  ║ 
q_4: ┤ X ├┤ H ├┤ X ├──╫───╫─┤ X ├──────╫──╫─
     └───┘└───┘└───┘  ║   ║ └───┘      ║  ║ 
c: 4/═════════════════╩═══╩════════════╩══╩═
                      1   3            0  2 
Результаты измерений: {'0101': 1024}


значение нечётных битов не имеет значения (от их изменения ничего не зависит а значит от них не зависит и тип функции). Поэтому результат 0101 можно интерпретировать как 11 - сбалансированная

Алгоритм Дойча–Йожи для f(x) = x ∨  ̅̅x , где ̅x = NOT x

| x | f(x) |
|---|------|
| 0 |   1  |
| 1 |   1  |

всегда добавляем фазу

In [10]:
def oracle_func_3(qc, n_qubits):
  qc.x(n_qubits)

deutsch_jozsa_algorithm(oracle_func_3, 1, "Алгоритм Дойча–Йожи для f(x) = x ∨  ̅̅x , где ̅x = NOT x")

Алгоритм Дойча–Йожи для f(x) = x ∨  ̅̅x , где ̅x = NOT x
     ┌───┐┌───┐     ┌─┐
q_0: ┤ H ├┤ H ├─────┤M├
     ├───┤├───┤┌───┐└╥┘
q_1: ┤ X ├┤ H ├┤ X ├─╫─
     └───┘└───┘└───┘ ║ 
c: 1/════════════════╩═
                     0 
Результаты измерений: {'0': 1024}


функция константная

Алгоритм Дойча–Йожи для f(x) = 1,если старший бит = 1

| x_0 | x_1 | x_2 | x_3 | f(x) |
|-----|-----|-----|-----|------|
|  ?  |  ?  |  ?  |  0  |   0  |
|  ?  |  ?  |  ?  |  1  |   1  |

добвляем фазу если старий бит == 1

In [11]:
def oracle_func_4(qc, n_qubits):
  qc.cx(n_qubits - 1, n_qubits)

deutsch_jozsa_algorithm(oracle_func_4, 4, "Алгоритм Дойча–Йожи для f(x) = x ∨  ̅̅x , где ̅x = NOT x")

Алгоритм Дойча–Йожи для f(x) = x ∨  ̅̅x , где ̅x = NOT x
     ┌───┐┌───┐     ┌─┐              
q_0: ┤ H ├┤ H ├─────┤M├──────────────
     ├───┤├───┤     └╥┘┌─┐           
q_1: ┤ H ├┤ H ├──────╫─┤M├───────────
     ├───┤├───┤      ║ └╥┘┌─┐        
q_2: ┤ H ├┤ H ├──────╫──╫─┤M├────────
     ├───┤└───┘      ║  ║ └╥┘┌───┐┌─┐
q_3: ┤ H ├───────■───╫──╫──╫─┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐ ║  ║  ║ └───┘└╥┘
q_4: ┤ X ├┤ H ├┤ X ├─╫──╫──╫───────╫─
     └───┘└───┘└───┘ ║  ║  ║       ║ 
c: 4/════════════════╩══╩══╩═══════╩═
                     0  1  2       3 
Результаты измерений: {'1000': 1024}


значение не старших битов не имеет значения (от их изменения ничего не зависит а значит от них не зависит и тип функции). Поэтому результат 1000 можно интерпретировать как 1 - сбалансированная

Алгоритм Дойча–Йожи для f(x) = x ⊕ x

| x | f(x) |
|---|------|
| 0 |   0  |
| 1 |   0  |

фазу добавлять не нужно ни при одном исходном значении

In [12]:
def oracle_func_5(qc, n_qubits):
  pass

deutsch_jozsa_algorithm(oracle_func_5, 1, "Алгоритм Дойча–Йожи для для f(x) = x ⊕ x")

Алгоритм Дойча–Йожи для для f(x) = x ⊕ x
     ┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ H ├┤M├
     ├───┤├───┤└╥┘
q_1: ┤ X ├┤ H ├─╫─
     └───┘└───┘ ║ 
c: 1/═══════════╩═
                0 
Результаты измерений: {'0': 1024}


функця константная

## Демонстрация квантового преимущества при n=25

Покажем на примере f(x) = (x_0  ⊕ x_2  ⊕ x_4  ⊕ ...) - XOR только чётных

In [13]:
n = 25

Реализация на кубитах

In [14]:
n_qubits = n

# n входных кубитов + 1 вспомогательный
qc = QuantumCircuit(n_qubits + 1, n_qubits)

# Шаг 1: Инициализация вспомогательного кубита в |1⟩
qc.x(n_qubits)

# Шаг 2: Применяем Адамары ко всем кубитам
for i in range(n_qubits + 1):
    qc.h(i)

# Шаг 3: Применяем оракул
oracle_func_2(qc, n_qubits)

# Шаг 4: Применяем Адамары ко входным кубитам
for i in range(n_qubits):
    qc.h(i)

# Шаг 5: Измеряем входные кубиты
for i in range(n_qubits):
    qc.measure(i, i)

simulate(qc, 1)

Результаты измерений: {'1010101010101010101010101': 1}


{'1010101010101010101010101': 1}

интерпретируем результат как '1111111111111' - функция сбалансированная

Классическая реализация

In [15]:
# f(x) = (x_0  ⊕ x_2  ⊕ x_4  ⊕ ...)
def f(x, n):
  res = 0
  for i in range(0, n + 1, 2):
    res ^= ((x >> i) & 0b1)

  return res

c_1 = 0
c_all = (1 << n)
for x in range(0, c_all):
  c_1 += f(x, n)

print("Всего единиц", c_1, "из", c_all)
if(c_1 * 2 == c_all):
  print("Функция сбалансированная")
elif(c_1 == c_all or c_1 == 0):
  print("Функция константная")
else:
  print("Не удалось определить вид функции")

Всего единиц 16777216 из 33554432
Функция сбалансированная


Как видно по времени запуска ячейки кода для классичского варианта потребовалось около 30 секунд, а для квантового меньше секунды.